# 영화추천시스템 만들기

In [3]:
import pandas as pd # pandas 라이브러리 
from sklearn.feature_extraction.text import TfidfVectorizer
# 추출시스템 from TfidVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# 코사인 유사도 예측 from 사이킷런 메트릭스 페어와이즈 

# pandas:
# pandas는 데이터 조작 및 분석에 사용되는 강력한 라이브러리입니다. 
# 데이터를 CSV, Excel, SQL 데이터베이스 등 다양한 형식에서 불러오고 처리하는 데 유용합니다.
# 영화 데이터를 불러오고 전처리하고 데이터 프레임 형태로 표시하는 데 효과적입니다. 
# 이로써 데이터를 쉽게 필터링하고 정리할 수 있습니다.

# TfidfVectorizer:
# TfidfVectorizer는 텍스트 데이터를 기계 학습 모델이 이해할 수 있는 수치 데이터로 변환하는 데 사용됩니다.
# 영화 설명과 같은 텍스트 데이터를 TF-IDF 가중치로 벡터화하여 각 문서(영화 설명)를 벡터로 표현합니다. 
# 이렇게 하면 영화 간의 유사성을 수치적으로 계산할 수 있습니다.

# cosine_similarity:
# cosine_similarity 함수는 벡터 간의 코사인 유사도를 계산하는 데 사용됩니다. 
# 코사인 유사도는 두 벡터 간의 방향 유사성을 측정하는 지표로, 특히 텍스트 데이터나 문서 간의 유사성을 계산하는 데 유용합니다.
# 영화 설명을 TF-IDF로 벡터화한 후, 이러한 벡터 간의 코사인 유사도를 계산하여 영화 간의 유사성을 파악하고 추천에 활용할 수 있습니다.

data = pd.read_csv('movies_metadata.csv', low_memory=False)
data.head(2)

# low_memory=False를 설정하면 pandas는 데이터 유형을 미리 확인하여 메모리 사용량을 늘리지만, 
# 데이터 형식을 정확하게 유추하므로 데이터 형식 관련 오류가 줄어듭니다. 
# 따라서 큰 CSV 파일을 처리할 때 데이터 유형을 정확하게 유추하려면 low_memory=False를 사용하는 것이 좋습니다.

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [4]:
df = data.head(20000)

In [5]:
df['overview'].isnull().sum()
# nono값 전부 카운팅

135

In [7]:
# 결측값을 빈 값으로 대체 ( copy()해서 사용하거나 다음처럼 사용)
df.loc[:, 'overview'] = df['overview'].fillna('')

# df.loc[:, 'overview']: df는 데이터프레임 객체를 나타내며, 
# .loc을 사용하여 데이터프레임 내의 특정 위치에 접근합니다.
# :은 모든 행을 선택하고, 'overview'는 'overview' 열을 선택합니다.

# df['overview'].fillna(''): 
# 이 부분은 'overview' 열을 선택하고, 
# .fillna('')를 사용하여 해당 열의 결측값을 빈 문자열('')로 대체합니다. 
#결측값이란 해당 셀에 아무 값도 없는 경우를 의미합니다.

# =: 결측값을 빈 문자열로 대체한 결과를 원래의 데이터프레임 열에 할당합니다. 
# 따라서 'overview' 열이 결측값을 빈 문자열로 채워진 새로운 열로 대체됩니다.

### scikit-learn : TfidfVectorizer()
CountVectorizer()+ TfidfTransformer()       
=> TfidfVectorizer() : 사용자의 편의를 위해 TF-IDF 변환을 빠르게 적용

In [8]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['overview'])
tfidf_matrix.shape

(20000, 47487)

In [9]:
# 200000개의 문서 벡터와 자기 자신을 포함한 20,000개의 문서 벡터간의 유사도 행렬 (즉, 각 영화의 개요에 대한 TF-IDF 벡터) 간의 코사인 유사도를 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :',cosine_sim.shape)

코사인 유사도 연산 결과 : (20000, 20000)


In [10]:
title_to_index = dict(zip(df['title'], df.index))

# 영화 제목 Father of the Bride Part II의 인덱스를 리턴
idx = title_to_index['Father of the Bride Part II']
print(idx)

4


In [11]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당 영화의 인덱스를 받아온다.
    idx = title_to_index[title]

    # 해당 영화와 모든 영화와의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아온다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 얻는다.
    movie_indices = [idx[0] for idx in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴한다.
    return df['title'].iloc[movie_indices]


In [12]:
# 다크 나이트 라이즈와 overview가 유사한 영화 찾기
get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object

### <유사도(cosine_similarity)를 이용한 영화 추천 시스템> 통합코드

In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

data = pd.read_csv('movies_metadata.csv', low_memory=False)
df = data.head(20000)
#df['overview'].isnull().sum()
df.loc[:, 'overview'] = df['overview'].fillna('')

# tfidf 구하기
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['overview'])

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

title_to_index = dict(zip(df['title'], df.index))

def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당 영화의 인덱스를 받아온다.
    idx = title_to_index[title]

    # 해당 영화와 모든 영화와의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아온다. 
    # 0번 인덱스에는 선택한 영화와의 유사도가 저장되어 있어 이를 제외하고 나머지 영화들(1번부터 10번까지)의 유사도를 가져옴
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 얻는다.
    movie_indices = [idx[0] for idx in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴한다.
    return df['title'].iloc[movie_indices]

get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object